**BÀI TẬP CHƯƠNG 3**

**Bài 1: Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B**

In [7]:
import sqlite3

# Tạo kết nối tới database SQLite (tạo mới hoặc mở database đã có)
conn = sqlite3.connect('samplee.db')
cursor = conn.cursor()

# Tạo bảng data_table với 2 cột A và B
cursor.execute('''
CREATE TABLE IF NOT EXISTS data_table (
    A REAL,
    B REAL
);
''')

# Chèn dữ liệu mẫu (nếu chưa có dữ liệu)
sample_data = [(2, 4), (3, 6), (5, 8), (7, 10), (9, 14)]
cursor.executemany('INSERT INTO data_table (A, B) VALUES (?, ?)', sample_data)

In [5]:
import math

# Kết nối database
conn = sqlite3.connect('samplee.db')

conn.create_function("SQRT", 1, math.sqrt)

cursor = conn.cursor()

# Truy vấn tính hệ số tương quan
query = '''
SELECT
    (COUNT(*) * SUM(A*B) - SUM(A)*SUM(B)) / (
        SQRT(COUNT(*)*SUM(A*A) - SUM(A)*SUM(A)) *
        SQRT(COUNT(*)*SUM(B*B) - SUM(B)*SUM(B))
    ) AS correlation_AB
FROM data_table;
'''

cursor.execute(query)
result = cursor.fetchone()

print("Hệ số tương quan giữa A và B:", result[0])

Hệ số tương quan giữa A và B: 0.9894382924043013


**Bài 2: Chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm định**

Chuyển đổi dữ liệu sang dạng quan hệ 

In [75]:
import sqlite3
import pandas as pd

# Kết nối database SQLite
conn = sqlite3.connect('car_scores.db')
cursor = conn.cursor()

# Tạo bảng car_scores
cursor.execute('''
CREATE TABLE IF NOT EXISTS car_scores (
    Day VARCHAR(10),
    Car VARCHAR(1),
    Score FLOAT
);
''')

# Chèn dữ liệu vào bảng
cursor.executemany('''
INSERT INTO car_scores (Day, Car, Score) VALUES (?, ?, ?)
''', [
    ('Day 1', 'A', 8),
    ('Day 1', 'B', 9),
    ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5),
    ('Day 2', 'B', 8.5),
    ('Day 2', 'C', 7),
    ('Day 3', 'A', 6),
    ('Day 3', 'B', 7),
    ('Day 3', 'C', 8),
    ('Day 4', 'A', 7),
    ('Day 4', 'B', 6),
    ('Day 4', 'C', 5)
])

# Thực hiện câu lệnh SELECT
df = pd.read_sql_query('SELECT * FROM car_scores', conn)
print(df)

conn.commit()

      Day Car  Score
0   Day 1   A    8.0
1   Day 1   B    9.0
2   Day 1   C    7.0
3   Day 2   A    7.5
4   Day 2   B    8.5
5   Day 2   C    7.0
6   Day 3   A    6.0
7   Day 3   B    7.0
8   Day 3   C    8.0
9   Day 4   A    7.0
10  Day 4   B    6.0
11  Day 4   C    5.0


In [77]:
# Kết nối database SQLite
conn = sqlite3.connect('car_scores.db')
cursor = conn.cursor()

# Thực hiện câu lệnh SELECT và tạo thêm cột Category
df_category = pd.read_sql_query('''
SELECT 
    Day,
    Car,
    Score,
    CASE 
        WHEN Score <= 6 THEN 'Low'
        WHEN Score <= 8 THEN 'Medium'
        ELSE 'High'
    END AS Category
FROM car_scores;
''', conn)
print("Dữ liệu sau khi thêm cột Category:")
print(df_category)

# Bảng tần suất của Car và Category 
df_car_freq = pd.read_sql_query('''
SELECT 
    Car,
    Category,
    COUNT(*) AS Frequency
FROM (
    SELECT 
        Car,
        CASE 
            WHEN Score <= 6 THEN 'Low'
            WHEN Score <= 8 THEN 'Medium'
            ELSE 'High'
        END AS Category
    FROM car_scores
) AS sub
GROUP BY Car, Category
ORDER BY Car, Category;
''', conn)
print("\nBảng tần suất Car và Category :")
print(df_car_freq)

# Bảng tần suất của Day và Category 
df_day_freq = pd.read_sql_query('''
SELECT 
    Day,
    CASE 
        WHEN Score <= 6 THEN 'Low'
        WHEN Score <= 8 THEN 'Medium'
        ELSE 'High'
    END AS Category,
    COUNT(*) AS Frequency
FROM car_scores
GROUP BY Day, Category
ORDER BY Day, Category;
''', conn)
print("\n Bảng tần suất Day và  Category:")
print(df_day_freq)

Dữ liệu sau khi thêm cột Category:
      Day Car  Score Category
0   Day 1   A    8.0   Medium
1   Day 1   B    9.0     High
2   Day 1   C    7.0   Medium
3   Day 2   A    7.5   Medium
4   Day 2   B    8.5     High
5   Day 2   C    7.0   Medium
6   Day 3   A    6.0      Low
7   Day 3   B    7.0   Medium
8   Day 3   C    8.0   Medium
9   Day 4   A    7.0   Medium
10  Day 4   B    6.0      Low
11  Day 4   C    5.0      Low

Bảng tần suất Car và Category :
  Car Category  Frequency
0   A      Low          1
1   A   Medium          3
2   B     High          2
3   B      Low          1
4   B   Medium          1
5   C      Low          1
6   C   Medium          3

 Bảng tần suất Day và  Category:
     Day Category  Frequency
0  Day 1     High          1
1  Day 1   Medium          2
2  Day 2     High          1
3  Day 2   Medium          2
4  Day 3      Low          1
5  Day 3   Medium          2
6  Day 4      Low          2
7  Day 4   Medium          1


Thực hiện kiểm định 

In [53]:
from scipy.stats import chi2_contingency

# Dữ liệu Car vs Category
car_data = pd.DataFrame({
    'Car': ['A', 'A', 'B', 'B', 'B', 'C', 'C'],
    'Category': ['Low', 'Medium', 'High', 'Low', 'Medium', 'Low', 'Medium'],
    'Frequency': [1, 3, 2, 1, 1, 1, 3]
})

car_table = car_data.pivot_table(index='Car', columns='Category', values='Frequency', fill_value=0)
print("Bảng tần suất (Car vs Category):\n", car_table)

# Thực hiện kiểm định Chi bình phương
chi2_car, p_car, dof_car, expected_car = chi2_contingency(car_table)
print("\nKết quả kiểm định Chi-square cho Car:")
print(f"Chi2 statistic = {chi2_car:.4f}")
print(f"p-value = {p_car:.4f}")

# Dữ liệu Day vs Category
day_data = pd.DataFrame({
    'Day': ['Day 1', 'Day 1', 'Day 2', 'Day 2', 'Day 3', 'Day 3', 'Day 4', 'Day 4'],
    'Category': ['High', 'Medium', 'High', 'Medium', 'Low', 'Medium', 'Low', 'Medium'],
    'Frequency': [1, 2, 1, 2, 1, 2, 2, 1]
})

day_table = day_data.pivot_table(index='Day', columns='Category', values='Frequency', fill_value=0)
print("\nBảng tần suất (Day vs Category):\n", day_table)

# Thực hiện kiểm định Chi bình phương
chi2_day, p_day, dof_day, expected_day = chi2_contingency(day_table)
print("\nKết quả kiểm định Chi-square cho Day:")
print(f"Chi2 statistic = {chi2_day:.4f}")
print(f"p-value = {p_day:.4f}")

Bảng tần suất (Car vs Category):
 Category  High  Low  Medium
Car                        
A          0.0  1.0     3.0
B          2.0  1.0     1.0
C          0.0  1.0     3.0

Kết quả kiểm định Chi-square cho Car:
Chi2 statistic = 5.1429
p-value = 0.2730

Bảng tần suất (Day vs Category):
 Category  High  Low  Medium
Day                        
Day 1      1.0  0.0     2.0
Day 2      1.0  0.0     2.0
Day 3      0.0  1.0     2.0
Day 4      0.0  2.0     1.0

Kết quả kiểm định Chi-square cho Day:
Chi2 statistic = 6.0952
p-value = 0.4126


- Dựa trên kiểm định Chi bình phương giữa các mẫu xe và phân loại điểm số, kết quả cho thấy không có sự khác biệt giữa các mẫu A, B, và C trong suốt 4 ngày thử nghiệm (vì p = 0.273 > 0.05).
- Kết quả thử nghiệm không phụ thuộc vào ngày cũng không phụ thuộc vào mẫu xe (vì cả hai p-value đều > 0.05)

**Bài 3: Chuyển đổi các giá trị này thành định dạng thời gian**

In [21]:
# Kết nối database SQLite
conn = sqlite3.connect('Fflights.db')
cursor = conn.cursor()

# Tạo bảng flights
cursor.execute('''
CREATE TABLE Fflights (
    id INT PRIMARY KEY,
    departure_time INT
);
''')

# Thêm dữ liệu mẫu
cursor.executemany('''
INSERT INTO Fflights (id, departure_time) VALUES (?, ?)''', [
    (1, 830),
    (2, 1445),
    (3, 30),
    (4, 0),
    (5, 2359)
])

# Hiển thị dữ liệu ban đầu
print("\nDữ liệu ban đầu:")
df_before = pd.read_sql_query('SELECT * FROM Fflights', conn)
print(df_before)

# Thêm cột departure_time_converted
cursor.execute('''
ALTER TABLE Fflights ADD COLUMN departure_time_converted TEXT;
''')

# Cập nhật giá trị cho cột mới
cursor.execute('''
UPDATE Fflights
SET departure_time_converted = 
  SUBSTR('0000' || departure_time, -4, 2) || ':' || SUBSTR('0000' || departure_time, -2, 2);
''')

# Hiển thị dữ liệu sau khi cập nhật
print("\nDữ liệu sau khi thêm departure_time_converted:")
df_after = pd.read_sql_query('SELECT * FROM Fflights', conn)
print(df_after)


Dữ liệu ban đầu:
   id  departure_time
0   1             830
1   2            1445
2   3              30
3   4               0
4   5            2359

Dữ liệu sau khi thêm departure_time_converted:
   id  departure_time departure_time_converted
0   1             830                    08:30
1   2            1445                    14:45
2   3              30                    00:30
3   4               0                    00:00
4   5            2359                    23:59


**Bài 4: Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD**

In [45]:
# Kết nối SQLite
conn = sqlite3.connect('SAless.db')
cursor = conn.cursor()

# Tạo bảng sales
cursor.execute('''
CREATE TABLE SAless (
    id INTEGER PRIMARY KEY,
    sale_amount INTEGER
);
''')

# Chèn dữ liệu
cursor.executemany('''
INSERT INTO SAless (id, sale_amount) VALUES (?, ?)''', [
    (1, 100),
    (2, 110),
    (3, 95),
    (4, 105),
    (5, 120),
    (6, 300),
    (7, 98),
    (8, 102)
])

# Hiển thị dữ liệu ban đầu
print("\nDữ liệu bảng Sales:")
df_sales = pd.read_sql_query('SELECT * FROM SAless', conn)
print(df_sales)

conn.commit()


Dữ liệu bảng Sales:
   id  sale_amount
0   1          100
1   2          110
2   3           95
3   4          105
4   5          120
5   6          300
6   7           98
7   8          102


In [47]:
# Tính median bằng SQL
query_median = '''
SELECT AVG(sale_amount) AS median
FROM (
  SELECT sale_amount
  FROM SAless
  ORDER BY sale_amount
  LIMIT 2 - (SELECT COUNT(*) FROM SAless) % 2
  OFFSET (SELECT (COUNT(*) - 1) / 2 FROM SAless)
);
'''
df_median = pd.read_sql_query(query_median, conn)
median = df_median["median"][0]
print("\nMedian:", median)


Median: 103.5


In [49]:
# Tính MAD bằng SQL
query_mad = f'''
SELECT AVG(abs_diff) AS mad
FROM (
  SELECT ABS(sale_amount - {median}) AS abs_diff
  FROM SAless
  ORDER BY abs_diff
  LIMIT CASE WHEN (SELECT COUNT(*) FROM SAless) % 2 = 0 THEN 2 ELSE 1 END
  OFFSET (SELECT (COUNT(*) - 1) / 2 FROM SAless)
);
'''

df_mad = pd.read_sql_query(query_mad, conn)
mad = df_mad["mad"][0]
print("MAD:", mad)

MAD: 6.0


In [51]:
# Tìm ngoại lệ
threshold = 1.5 * mad
query_outliers = f'''
SELECT *
FROM SAless
WHERE ABS(sale_amount - {median}) > {threshold};
'''
df_outliers = pd.read_sql_query(query_outliers, conn)
print("\nCác giá trị ngoại lệ (dựa trên 1.5 * MAD):")
print(df_outliers)


Các giá trị ngoại lệ (dựa trên 1.5 * MAD):
   id  sale_amount
0   5          120
1   6          300


**Bài 5: Xác định liệu 2 người trong bảng patient có phải là một người hay không?**

In [39]:
# Kết nối SQLite
conn = sqlite3.connect('patients.db')
cursor = conn.cursor()

# Tạo bảng Patient
cursor.execute('''
CREATE TABLE patients (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight INTEGER,
    height INTEGER
);
''')

# Chèn dữ liệu mẫu
cursor.executemany('''
INSERT INTO patients (id, last_name, weight, height) VALUES (?, ?, ?, ?)''', [
    (1, 'Nguyen', 60, 165),
    (2, 'Nguyen', 61, 165),
    (3, 'Tran', 55, 160),
    (4, 'Nguyen', 72, 170),
    (5, 'Tran', 55, 159),
    (6, 'Le', 65, 168)
])

# In bảng dữ liệu mẫu
print("\nDữ liệu mẫu từ bảng Patients:")
df_patients = pd.read_sql_query('SELECT * FROM patients', conn)
print(df_patients)

# Truy vấn tính điểm Boolean
query = '''
SELECT 
    p1.id AS id1, p2.id AS id2,
    p1.last_name AS name1, p2.last_name AS name2,
    p1.weight AS weight1, p2.weight AS weight2,
    
    (CASE WHEN p1.last_name = p2.last_name THEN 1 ELSE 0 END) +
    (CASE WHEN ABS(p1.weight - p2.weight) <= 2 THEN 1 ELSE 0 END) AS match_score

FROM patients p1
JOIN patients p2 ON p1.id < p2.id
WHERE
    (p1.last_name = p2.last_name) AND (ABS(p1.weight - p2.weight) <= 2);
'''

df_matches = pd.read_sql_query(query, conn)
print("\nCác cặp bệnh nhân có khả năng là cùng người:")
print(df_matches)


Dữ liệu mẫu từ bảng Patients:
   id last_name  weight  height
0   1    Nguyen      60     165
1   2    Nguyen      61     165
2   3      Tran      55     160
3   4    Nguyen      72     170
4   5      Tran      55     159
5   6        Le      65     168

Các cặp bệnh nhân có khả năng là cùng người:
   id1  id2   name1   name2  weight1  weight2  match_score
0    1    2  Nguyen  Nguyen       60       61            2
1    3    5    Tran    Tran       55       55            2
